In [1]:
!pip install git+https://github.com/GerbenBeintema/deepSI@master

  Cloning https://github.com/GerbenBeintema/deepSI (to revision master) to /tmp/pip-req-build-nllqn0wv
  Running command git clone --filter=blob:none --quiet https://github.com/GerbenBeintema/deepSI /tmp/pip-req-build-nllqn0wv
  Resolved https://github.com/GerbenBeintema/deepSI to commit 28c96c174fa2e1c83aeb26091d67785d468a4bee
  Preparing metadata (setup.py) ... done


In [2]:
import deepSI
from deepSI import System_data, System_data_list
import numpy as np
from matplotlib import pyplot as plt
import torch
import cv2
from torch import optim, nn
from tqdm.auto import tqdm
import matplotlib
from torch.utils.data import Dataset, DataLoader

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
linewidth = 3.320
reducesize = 0.85
fig_fontsize = (10*19/28)*reducesize
plt.rcParams.update({'font.size': fig_fontsize})
dpi = 200
pad = 0.4

In [3]:
archive_path = 'ffmpeg-master-latest-linux64-gpl.tar.xz'
output_dir = 'ffmpeg/'  # You can use an existing or new directory

# Extract the archive
!tar -xf {archive_path} -C {output_dir}

In [4]:
import os

# Example directory containing the FFmpeg binaries
ffmpeg_dir = 'ffmpeg/ffmpeg-master-latest-linux64-gpl/bin'

# Add the FFmpeg directory to the PATH environment variable
os.environ['PATH'] = ffmpeg_dir + os.pathsep + os.environ['PATH']

In [5]:
# Load the npz file
adi = 12
systemsdir = 'systems_force/'
filen = 'combined_output_20_3.npz'
data = np.load(filen, allow_pickle=True)

# Assuming you have arrays 'frames' and 'forces' within your npz file
frames = data['frames']
forces = data['forces']

# Determine the size of each set
total_size = len(frames)
train_size = int(total_size * 0.6)  # 50% of the data for training
val_size = int(total_size * 0.20)  # 20% of the data for validation
test_size = total_size - train_size - val_size  # Remaining 30% for testing

# Function to resize and reshape frames with progress bar
def resize_and_reshape_frames(frames, batch_size, new_height, new_width):
    num_frames = frames.shape[0]
    resized_and_reshaped_frames = np.zeros((num_frames, frames.shape[3], new_height, new_width), dtype=np.uint8)
    for start in tqdm(range(0, num_frames, batch_size), desc="Resizing frames"):
        end = start + batch_size
        batch_frames = frames[start:end]
        for i in range(batch_frames.shape[0]):
            resized_frame = cv2.resize(batch_frames[i], (new_width, new_height), interpolation=cv2.INTER_LINEAR)
            resized_and_reshaped_frames[start + i] = resized_frame.transpose(2, 0, 1)
    return resized_and_reshaped_frames

# Resize and reshape parameters
new_height = frames.shape[1] // 2
new_width = frames.shape[2] // 2
batch_size = 30

# Resize and reshape all frames
frames_resized_reshaped = resize_and_reshape_frames(frames, batch_size, new_height, new_width).astype(np.float32)

Resizing frames:   0%|          | 0/400 [00:00<?, ?it/s]

In [6]:
# Split the data
del frames
frames_train = frames_resized_reshaped[:train_size]
frames_val = frames_resized_reshaped[train_size:train_size + val_size]
frames_test = frames_resized_reshaped[train_size + val_size:]
n_channels, height, width = frames_train.shape[1], frames_train.shape[2], frames_train.shape[3]


In [7]:
forces_train = forces[:train_size, 2]
forces_val = forces[train_size:train_size + val_size, 2]
forces_test = forces[train_size + val_size:, 2]
del forces
# Initialize the SS_encoder_CNN_video system
#sys_vbss = deepSI.fit_systems.SS_encoder_CNN_video(na=adi, nb=adi)
n_channels, height, width = frames_train.shape[1], frames_train.shape[2], frames_train.shape[3]
#sys_vbss.init_nets(nu=1, ny=(n_channels, height, width))

In [8]:
# Create System_data instances with resized and reshaped frames
system_data = System_data(y=forces_train, u=frames_train)
system_data_val = System_data(y=forces_val, u=frames_val)
system_data_test = System_data(y=forces_test, u=frames_test)
del forces_train, frames_train, forces_val, frames_val, forces_test, frames_test

In [9]:
from deepSI.utils.torch_nets import CNN_encoder


#Psi(upast, ypast) where upast is an image and ypast an vector
#nb, nu, na, ny, nx
class CNN_encoder_image_input(nn.Module):
    def __init__(self, nb, nu, na, ny, nx, n_nodes_per_layer=64, n_hidden_layers=2, activation=nn.Tanh, features_ups_factor=1.33):
        super(CNN_encoder_image_input, self).__init__()
        self.net = CNN_encoder(na, ny, nb, nu, nx, \
                               n_nodes_per_layer=n_nodes_per_layer, \
                               n_hidden_layers=n_hidden_layers, \
                                activation=activation, \
                                features_ups_factor=features_ups_factor)

    def forward(self, upast, ypast):
        return self.net(ypast, upast)

#nx, nu
class CNN_encoder_f_image(nn.Module):
    def __init__(self, nx, nu, n_nodes_per_layer=64, n_hidden_layers=2, activation=nn.Tanh, features_ups_factor=1.33):
        super(CNN_encoder_f_image, self).__init__()
        self.net = CNN_encoder(1, nx, 1, nu, nx, \
                               n_nodes_per_layer=n_nodes_per_layer, \
                               n_hidden_layers=n_hidden_layers, \
                                activation=activation, \
                                features_ups_factor=features_ups_factor)

    def forward(self, x, u):
        #x.shape = (Nbatch, nx)
        #u.shape = (Nbatch, C, H, W)

        #self.net 
        # x -> upast = (Nbatch, nb, nu)
        # u -> ypast = (Nbatch, na, C, H, W)
        return self.net(x[:,None], u[:,None])
    
#nx, ny, nu=-1
class CNN_encoder_h_image(nn.Module):
    def __init__(self, nx, ny, nu, n_nodes_per_layer=64, n_hidden_layers=2, activation=nn.Tanh, features_ups_factor=1.33):
        super(CNN_encoder_h_image, self).__init__()
        self.net = CNN_encoder(1, nx, 1, nu, ny, \
                               n_nodes_per_layer=n_nodes_per_layer, \
                               n_hidden_layers=n_hidden_layers, \
                                activation=activation, \
                                features_ups_factor=features_ups_factor)

    def forward(self, x, u):
        #x.shape = (Nbatch, nx)
        #u.shape = (Nbatch, C, H, W)

        #self.net 
        # x -> upast = (Nbatch, nb, nu)
        # u -> ypast = (Nbatch, na, C, H, W)
        return self.net(x[:,None], u[:,None])



In [10]:
# from deepSI.utils import CNN_chained_upscales, CNN_encoder
from deepSI.fit_systems import SS_encoder_general

class SS_encoder_CNN_video_input(SS_encoder_general):
    """The subspace encoder convolutonal neural network with image inputs

    Notes
    -----
    The subspace encoder

    """
    def __init__(self, nx=10, na=20, nb=20, feedthrough=True, e_net=CNN_encoder_image_input, \
                 f_net=CNN_encoder_f_image, h_net=CNN_encoder_h_image, \
                                            e_net_kwargs={}, f_net_kwargs={}, h_net_kwargs={}):
        super(SS_encoder_CNN_video_input, self).__init__(nx=nx,na=na,nb=nb, feedthrough=feedthrough, \
            e_net=e_net,               f_net=f_net,                h_net=h_net, \
            e_net_kwargs=e_net_kwargs, f_net_kwargs=f_net_kwargs,  h_net_kwargs=h_net_kwargs)

model = SS_encoder_CNN_video_input(nx=20, na=12, nb=12)

# nu = (3,25,30)
# ny = 1
# N = 200
# u_seq = np.random.randn(N,*nu)
# y_seq = np.random.randn(N, ny)
#sys_data = System_data(u_seq, y_seq)

print(system_data)
model.init_model(nu=(3,135,240), ny=1)


System_data of length: 7200 nu=(3, 135, 240) ny=None normed=False dt=None


In [14]:
def fit(self, train_sys_data, val_sys_data, epochs=30, batch_size=256, loss_kwargs={}, \
            auto_fit_norm=True, validation_measure='sim-NRMS', optimizer_kwargs={}, concurrent_val=False, cuda=False, \
            timeout=None, verbose=1, sqrt_train=True, num_workers_data_loader=0, print_full_time_profile=False, scheduler_kwargs={}):
        '''The batch optimization method with parallel validation, 

        Parameters
        ----------
        train_sys_data : System_data or System_data_list
            The system data to be fitted
        val_sys_data : System_data or System_data_list
            The validation system data after each used after each epoch for early stopping. Use the keyword argument validation_measure to specify which measure should be used. 
        epochs : int
        batch_size : int
        loss_kwargs : dict
            The Keyword Arguments to be passed to the self.make_training_data and self.loss of the current fit_system.
        auto_fit_norm : boole
            If true will use self.norm.fit(train_sys_data) which will fit it element wise. 
        validation_measure : str
            Specify which measure should be used for validation, e.g. 'sim-RMS', '10-step-last-RMS', 'sim-NRMS_sys_norm', ect. See self.cal_validation_error for details.
        optimizer_kwargs : dict
            The Keyword Arguments to be passed on to init_optimizer. notes; init_optimizer['optimizer'] is the optimization function used (default torch.Adam)
            and optimizer_kwargs['parameters_optimizer_kwargs'] the learning rates and such for the different elements of the models. see https://pytorch.org/docs/stable/optim.html
        concurrent_val : boole
            If set to true a subprocess will be started which concurrently evaluates the validation method selected.
            Warning: if concurrent_val is set than "if __name__=='__main__'" or import from a file if using self defined method or networks.
        cuda : bool
            if cuda will be used (often slower than not using it, be aware)
        timeout : None or number
            Alternative to epochs to run until a set amount of time has past. 
        verbose : int
            Set to 0 for a silent run
        sqrt_train : boole
            will sqrt the loss while printing
        num_workers_data_loader : int
            see https://pytorch.org/docs/stable/data.html
        print_full_time_profile : boole
            will print the full time profile, useful for debugging and basic process optimization. 
        scheduler_kwargs : dict
            learning rate scheduals are a work in progress.
        
        Notes
        -----
        This method implements a batch optimization method in the following way; each epoch the training data is scrambled and batched where each batch
        is passed to the self.loss method and utilized to optimize the parameters. After each epoch the systems is validated using the evaluation of a 
        simulation or a validation split and a checkpoint will be crated if a new lowest validation loss has been achieved. (or concurrently if concurrent_val=True)
        After training (which can be stopped at any moment using a KeyboardInterrupt) the system is loaded with the lowest validation loss. 

        The default checkpoint location is "C:/Users/USER/AppData/Local/deepSI/checkpoints" for windows and ~/.deepSI/checkpoints/ for unix like.
        These can be loaded manually using sys.load_checkpoint("_best") or "_last". (For this to work the sys.unique_code needs to be set to the correct string)
        '''
        def validation(train_loss=None, time_elapsed_total=None):
            self.eval(); self.cpu()
            Loss_val = self.cal_validation_error(val_sys_data, validation_measure=validation_measure)
            self.Loss_val.append(Loss_val)
            self.Loss_train.append(train_loss)
            self.time.append(time_elapsed_total)
            self.batch_id.append(self.batch_counter)
            self.epoch_id.append(self.epoch_counter)
            if self.bestfit>=Loss_val:
                self.bestfit = Loss_val
                self.checkpoint_save_system()
            if cuda: 
                self.cuda()
            self.train()
            return Loss_val
        
        ########## Initialization ##########
        if self.init_model_done==False:
            if verbose: print('Initilizing the model and optimizer')
            device = 'cuda' if cuda else 'cpu'
            optimizer_kwargs = deepcopy(optimizer_kwargs)
            parameters_optimizer_kwargs = optimizer_kwargs.get('parameters_optimizer_kwargs',{})
            if parameters_optimizer_kwargs:
                del optimizer_kwargs['parameters_optimizer_kwargs']
            self.init_model(sys_data=train_sys_data, device=device, auto_fit_norm=auto_fit_norm, optimizer_kwargs=optimizer_kwargs,\
                    parameters_optimizer_kwargs=parameters_optimizer_kwargs, scheduler_kwargs=scheduler_kwargs)
        else:
            if verbose: print('Model already initilized (init_model_done=True), skipping initilizing of the model, the norm and the creation of the optimizer')
            self._check_and_refresh_optimizer_if_needed() 


        if self.scheduler==False and verbose:
            print('!!!! Your might be continuing from a save which had scheduler but which was removed during saving... check this !!!!!!')
        
        self.dt = train_sys_data.dt
        if cuda: 
            self.cuda()
        self.train()

        self.epoch_counter = 0 if len(self.epoch_id)==0 else self.epoch_id[-1]
        self.batch_counter = 0 if len(self.batch_id)==0 else self.batch_id[-1]
        extra_t            = 0 if len(self.time)    ==0 else self.time[-1] #correct timer after restart

        ########## Getting the data ##########
        data_train = self.make_training_data(self.norm.transform(train_sys_data), **loss_kwargs)
        if not isinstance(data_train, Dataset) and verbose: print_array_byte_size(sum([d.nbytes for d in data_train]))

        #### transforming it back to a list to be able to append. ########
        self.Loss_val, self.Loss_train, self.batch_id, self.time, self.epoch_id = list(self.Loss_val), list(self.Loss_train), list(self.batch_id), list(self.time), list(self.epoch_id)

        #### init monitoring values ########
        Loss_acc_val, N_batch_acc_val, val_counter, best_epoch, batch_id_start = 0, 0, 0, 0, self.batch_counter #to print the frequency of the validation step.
        N_training_samples = len(data_train) if isinstance(data_train, Dataset) else len(data_train[0])
        batch_size = min(batch_size, N_training_samples)
        N_batch_updates_per_epoch = N_training_samples//batch_size
        if verbose>0: 
            print(f'N_training_samples = {N_training_samples}, batch_size = {batch_size}, N_batch_updates_per_epoch = {N_batch_updates_per_epoch}')
        
        ### convert to dataset ###
        if isinstance(data_train, Dataset):
            persistent_workers = False if num_workers_data_loader==0 else True
            data_train_loader = DataLoader(data_train, batch_size=batch_size, drop_last=True, shuffle=True, \
                                   num_workers=num_workers_data_loader, persistent_workers=persistent_workers)
        else: #add my basic DataLoader
            data_train_loader = My_Simple_DataLoader(data_train, batch_size=batch_size) #is quite a bit faster for low data situations

        if concurrent_val:
            self.remote_start(val_sys_data, validation_measure)
            self.remote_send(float('nan'), extra_t)
        else: #start with the initial validation 
            validation(train_loss=float('nan'), time_elapsed_total=extra_t) #also sets current model to cuda
            if verbose: 
                print(f'Initial Validation {validation_measure}=', self.Loss_val[-1])

        try:
            t = Tictoctimer()
            start_t = time.time() #time keeping
            epochsrange = range(epochs) if timeout is None else itertools.count(start=0)
            if timeout is not None and verbose>0: 
                print(f'Starting indefinite training until {timeout} seconds have passed due to provided timeout')

            for epoch in (tqdm(epochsrange) if verbose>0 else epochsrange):
                bestfit_old = self.bestfit #to check if a new lowest validation loss has been achieved
                Loss_acc_epoch = 0.
                t.start()
                t.tic('data get')
                for train_batch in data_train_loader:
                    if cuda:
                        train_batch = [b.cuda() for b in train_batch]
                    t.toc('data get')
                    def closure(backward=True):
                        t.toc('optimizer start')
                        t.tic('loss')
                        Loss = self.loss(*train_batch, **loss_kwargs)
                        t.toc('loss')
                        if backward:
                            t.tic('zero_grad')
                            self.optimizer.zero_grad()
                            t.toc('zero_grad')
                            t.tic('backward')
                            Loss.backward()
                            t.toc('backward')
                        t.tic('stepping')
                        return Loss

                    t.tic('optimizer start')
                    training_loss = self.optimizer.step(closure).item()
                    t.toc('stepping')
                    if self.scheduler:
                        t.tic('scheduler')
                        self.scheduler.step()
                        t.tic('scheduler')
                    Loss_acc_val += training_loss
                    Loss_acc_epoch += training_loss
                    N_batch_acc_val += 1
                    self.batch_counter += 1
                    self.epoch_counter += 1/N_batch_updates_per_epoch

                    t.tic('val')
                    if concurrent_val and self.remote_recv(): ####### validation #######
                        self.remote_send(Loss_acc_val/N_batch_acc_val, time.time()-start_t+extra_t)
                        Loss_acc_val, N_batch_acc_val, val_counter = 0., 0, val_counter + 1
                    t.toc('val')
                    t.tic('data get')
                t.toc('data get')

                ########## end of epoch clean up ##########
                train_loss_epoch = Loss_acc_epoch/N_batch_updates_per_epoch
                if np.isnan(train_loss_epoch):
                    if verbose>0: print(f'&&&&&&&&&&&&& Encountered a NaN value in the training loss at epoch {epoch}, breaking from loop &&&&&&&&&&')
                    break

                t.tic('val')
                if not concurrent_val:
                    validation(train_loss=train_loss_epoch, \
                               time_elapsed_total=time.time()-start_t+extra_t) #updates bestfit and goes back to cpu and back
                t.toc('val')
                t.pause()

                ######### Printing Routine ##########
                if verbose>0:
                    time_elapsed = time.time() - start_t
                    if bestfit_old > self.bestfit:
                        print(f'########## New lowest validation loss achieved ########### {validation_measure} = {self.bestfit}')
                        best_epoch = epoch+1
                    if concurrent_val: #if concurrent val than print validation freq
                        val_feq = val_counter/(epoch+1)
                        valfeqstr = f', {val_feq:4.3} vals/epoch' if (val_feq>1 or val_feq==0) else f', {1/val_feq:4.3} epochs/val'
                    else: #else print validation time use
                        valfeqstr = f''
                    trainstr = f'sqrt loss {train_loss_epoch**0.5:7.4}' if sqrt_train and train_loss_epoch>=0 else f'loss {train_loss_epoch:7.4}'
                    Loss_val_now = self.Loss_val[-1] if len(self.Loss_val)!=0 else float('nan')

                    # Handle validation loss as float
                    if isinstance(Loss_val_now, (list, np.ndarray)) and len(Loss_val_now) == 1:
                        Loss_val_now = Loss_val_now[0]

                    Loss_str = f'Epoch {epoch+1:4}, {trainstr}, Val {validation_measure} {Loss_val_now:6.4f}'
                    loss_time = (t.acc_times['loss'] + t.acc_times['optimizer start'] + t.acc_times['zero_grad'] + t.acc_times['backward'] + t.acc_times['stepping'])  / t.time_elapsed
                    time_str = f'Time Loss: {loss_time:.1%}, data: {t.acc_times["data get"]/t.time_elapsed:.1%}, val: {t.acc_times["val"]/t.time_elapsed:.1%}{valfeqstr}'

                    self.batch_feq = (self.batch_counter - batch_id_start)/(time.time() - start_t)
                    batch_str = (f'{self.batch_feq:4.1f} batches/sec' if (self.batch_feq>1 or self.batch_feq==0) else f'{1/self.batch_feq:4.1f} sec/batch')
                    print(f'{Loss_str}, {time_str}, {batch_str}')
                    if print_full_time_profile:
                        print('Time profile:',t.percent())

                ####### Timeout Breaking ##########
                if timeout is not None:
                    if time.time() >= start_t+timeout:
                        break
        except KeyboardInterrupt:
            print('Stopping early due to a KeyboardInterrupt')

        self.train(); self.cpu()
        del data_train_loader

        ####### end of training concurrent things #####
        if concurrent_val:
            if verbose: print(f'Waiting for started validation process to finish and one last validation... (receiving = {self.remote.receiving})',end='')
            if self.remote_recv(wait=True):
                if verbose: print('Recv done... ',end='')
                if N_batch_acc_val>0:
                    self.remote_send(Loss_acc_val/N_batch_acc_val, time.time()-start_t+extra_t)
                    self.remote_recv(wait=True)
            self.remote_close()
            if verbose: print('Done!')

        
        self.Loss_val, self.Loss_train, self.batch_id, self.time, self.epoch_id = np.array(self.Loss_val), np.array(self.Loss_train), np.array(self.batch_id), np.array(self.time), np.array(self.epoch_id)
        self.checkpoint_save_system(name='_last')
        try:
            self.checkpoint_load_system(name='_best')
        except FileNotFoundError:
            print('no best checkpoint found keeping last')
        if verbose: 
            print(f'Loaded model with best known validation {validation_measure} of {self.bestfit:6.4} which happened on epoch {best_epoch} (epoch_id={self.epoch_id[-1] if len(self.epoch_id)>0 else 0:.2f})')

deepSI.fit_systems.fit = corrected_fit

In [15]:
def get_base_results(load=True, timeout=10000,n_channels=n_channels, height=height, width=width):
    train, val, test = system_data, system_data_val, system_data_test
    print(train)
    if load:
        sys_vbss_s = deepSI.load_system(systemsdir+'sse-cnn-base-last')
        sys_vbss_s._dt = None
        #sys_vbss_s.feedthrough=False
        sys_vbss_s.norm.u0 = np.mean(train.u, axis=(0, 2, 3))[:, None, None]
        sys_vbss_s.norm.ustd = np.std(train.u, axis=(0, 2, 3))[:, None, None]
        ##Normalize forces by computing mean and standard deviation over samples
        sys_vbss_s.norm.y0 = np.mean(train.y, axis=0)
        sys_vbss_s.norm.ystd = np.std(train.y, axis=0)
        print(sys_vbss_s.norm)
        sys_vbss_best = deepSI.load_system(systemsdir+'sse-cnn-base-best')
        sys_vbss_best._dt = None
        sys_vbss_best.feedthrough=False
        sys_vbss_t = sys_vbss_best.apply_experiment(test).NRMS(test)
    else:
        
        sys_vbss_s = SS_encoder_CNN_video_input(nx=20, na=12, nb=12)
        sys_vbss_s.init_model(nu=(3,135,240), ny=1)
        sys_vbss_s.feedthrough=True
        ##Normalize the frames by computing mean and standard deviation over samples, height, and width
        sys_vbss_s.norm.u0 = np.mean(train.u, axis=(0, 2, 3))[:, None, None]
        sys_vbss_s.norm.ustd = np.std(train.u, axis=(0, 2, 3))[:, None, None]
        ##Normalize forces by computing mean and standard deviation over samples
        sys_vbss_s.norm.y0 = np.mean(train.y, axis=0)
        sys_vbss_s.norm.ystd = np.std(train.y, axis=0)
        print(sys_vbss_s.norm)
        ## n_channels, height, width = frames_train.shape[1], frames_train.shape[2], frames_train.shape[3]
        sys_vbss_s.fit(system_data, val_sys_data=system_data_val, cuda=True, 
                       epochs=round(1500/(2*adi)), timeout=timeout, 
                       batch_size=128, 
                       validation_measure='sim-NRMS_sys_norm', 
                       loss_kwargs={'online_construct': True, 'nf':5},
                       auto_fit_norm=False,
                       #optimizer_kwargs={'lr':5e-4}
                      )
        sys_vbss_s.save_system('sse-cnn-base-force-best')
        sys_vbss_t = sys_vbss_s.apply_experiment(test).NRMS(test)
        sys_vbss_s.checkpoint_load_system('_last')
        sys_vbss_s.save_system('sse-cnn-base-force-last')

    return sys_vbss_s, sys_vbss_t

sys_vbss = get_base_results(load=False) 

System_data of length: 7200 nu=(3, 135, 240) ny=None normed=False dt=None
System_data_norm: (u0=[[[179.06047]]

 [[178.54716]]

 [[174.84698]]], ustd=[[[81.04006 ]]

 [[81.9512  ]]

 [[88.001884]]], y0=-2.81599406957767, ystd=2.198542996611473)
Model already initilized (init_model_done=True), skipping initilizing of the model, the norm and the creation of the optimizer
N_training_samples = 7184, batch_size = 128, N_batch_updates_per_epoch = 56
Initial Validation sim-NRMS_sys_norm= [0.96985066]
Starting indefinite training until 10000 seconds have passed due to provided timeout


0it [00:00, ?it/s]

TypeError: unsupported format string passed to numpy.ndarray.__format__

In [ ]:
t = lambda x: torch.as_tensor(x, dtype=torch.float32)

upast, ypast, ufuture, yfuture = [t(x) for x in model.make_training_data(model.norm.transform(sys_data), nf=4)]

In [ ]:
print(f'{upast.shape=}')
print(f'{ypast.shape=}')
print(f'{ufuture.shape=}')
print(f'{yfuture.shape=}')

xinit = model.encoder(upast, ypast)

In [ ]:
xnext = model.fn(xinit,ufuture[:,0])

In [ ]:
ynext = model.hn(xnext,ufuture[:,1])


In [ ]:
ynext.shape